In [36]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import sys
import timeit
import heapq 

In [37]:
resultA = []
resultB = []

In [38]:
class Graph():
    
    def __init__(self, vertices):
        self.V = vertices
        self.matrixgraph = []
        self.listgraph = {}
    
    def generateNew(self):
        self.matrixgraph = np.random.randint(0,10, (self.V, self.V)) #Generate random adj matrix + set limit of weights
        
        for i in range(self.V):                                     #Set diagonal of adj matrix to 0, this is to eliminate loops
            for j in range(self.V):
                if i == j:
                    self.matrixgraph[i][j] = 0
                    
        for i in range(2, self.V):               # Assumes that node 0 is our source node, eliminates all other edges except
            self.matrixgraph[0][i] = 0           # the edge from node 0 -> node 1.
        for i in range(1, self.V):
            self.matrixgraph[i][0] = 0
        
        if self.matrixgraph[0][1] == 0:          # Incase of bad RNG, form connection from source to node 1
            self.matrixgraph[0][1] = 1
            
        self.convertToAdj()                                         #Convert from Adj matrix to Adj list
                    
    def getMatrix(self):                                            #Prints the adj matrix
        print("Adjacency Matrix for the graph,")
        print(g.matrixgraph)
    
    def getList(self):                                              #Prints the adj list
        print("Adjacency List for the graph,")
        for nodes in self.listgraph:
            print(nodes, end = "")
            for edges in self.listgraph[nodes]:
                print(" -> ", edges[0], "Weight:", edges[1], end = "")
            print()
            
    def visualize(self):                                            #Visualize the graph
        G = nx.from_numpy_matrix(np.matrix(g.matrixgraph), create_using=nx.DiGraph)
        layout = nx.spring_layout(G)
        nx.draw(G, layout, with_labels=True)
        labels = nx.get_edge_attributes(G, "weight")
        nx.draw_networkx_edge_labels(G, pos=layout, edge_labels=labels)
        plt.show()
        
    def convertToAdj(self):
        for i in range(self.V):                                     #Creation of adj list using adj matrix
            self.listgraph[i] = []
        for i in range(self.V):
            for j in range(self.V):
                if self.matrixgraph[i][j] != 0:
                    temp = [j, self.matrixgraph[i][j]]
                    self.listgraph[i].append(temp)

In [39]:
g = Graph(50)
g.generateNew()
g.getMatrix()
print()
# g.getList()
# print()
# g.visualize()

Adjacency Matrix for the graph,
[[0 5 0 ... 0 0 0]
 [0 0 3 ... 9 9 4]
 [0 8 0 ... 5 2 1]
 ...
 [0 4 6 ... 0 7 6]
 [0 3 7 ... 0 0 9]
 [0 6 8 ... 0 9 0]]



Part (a) - Implementation of Dijkstra's algorithm using adjacency matrix + array for priority queue

In [40]:
class Node():
    
    def __init__(self, vertex, weight):
        self.vertex = vertex
        self.weight = weight

In [41]:
class PriorityQueue:

    def __init__(self):
        self.q = []

    def insert(self, node):
        if len(self.q) == 0:
            self.q.append(node)
        else:
            for i in range(len(self.q)):
                if node.weight < self.q[i].weight:
                    self.q.insert(i, node)
                    break
                elif node.weight == self.q[i].weight:
                    pass

                if i == len(self.q) - 1:
                    self.q.insert(i + 1, node)

    def getMinNode(self):
        return self.q[0].vertex

    def getMinWeight(self):
        return self.q[0].weight

    def pop(self):
        self.q.pop(0)

    def show(self):
        print("Current priority queue,")
        for i in range(len(self.q)):
            print(self.q[i].vertex, "- Weight", self.q[i].weight)

    def isEmpty(self):
        return len(self.q) == 0

    def removeNode(self, index):
        for i in range(len(self.q)):
            if self.q[i].vertex == index:
                self.q.pop(i)
                break

In [42]:
def DijkstraA(G, source):
    time_start = timeit.default_timer()
    
    distance = [sys.maxsize] * G.V  # Original distance for all vertices are set to infinity
    path = [[] for i in range(G.V)]  # Initialize N number of sub-lists to store the path to each node
    S = [False] * G.V  # Initialize False to all the nodes since shortest path not yet discovered

    distance[source] = 0  # Set distance of source node to be zero
    pQueue = PriorityQueue()

    for i in range(G.V):  # Insert all the vertices into priority queue
        tempNode = Node(i, distance[i])
        pQueue.insert(tempNode)

    while (not pQueue.isEmpty()):
        u = pQueue.getMinNode()
        pQueue.pop()
        S[u] = True

        for i in range(G.V):
            if G.matrixgraph[u][i] != 0:
                if S[i] != True and distance[i] > distance[u] + G.matrixgraph[u][i]:
                    pQueue.removeNode(i)
                    distance[i] = distance[u] + G.matrixgraph[u][i]                   
                    if source not in path[i]:
                        for elements in path[u]:
                            path[i].append(elements)
                    path[i].append(u)
                    pQueue.insert(Node(i, distance[i]))
    
    time_stop = timeit.default_timer()
    #END OF DIJKSTRA ALGORITHM
    
    #Print results
    print("Time elapsed for program: ", (time_stop - time_start)*1000, "milliseconds")  # Time in milliseconds
    print("Distance from source node,")
    for i in range(0, G.V):
        if distance[i] == sys.maxsize:
            print("Source to node", i, ": No such path exist")
        else:
            print("Source to node", i, ":", distance[i])
        resultA.append(distance[i])
    
    print()
    print("Path to get to each node,")
    
    for i in range(0, G.V):
        print("Path from source to node", i)
        print(path[i])
        print()

In [43]:
DijkstraA(g, 0)

Time elapsed for program:  4.629299999919567 milliseconds
Distance from source node,
Source to node 0 : 0
Source to node 1 : 5
Source to node 2 : 8
Source to node 3 : 7
Source to node 4 : 7
Source to node 5 : 7
Source to node 6 : 7
Source to node 7 : 8
Source to node 8 : 6
Source to node 9 : 7
Source to node 10 : 9
Source to node 11 : 7
Source to node 12 : 6
Source to node 13 : 8
Source to node 14 : 8
Source to node 15 : 7
Source to node 16 : 8
Source to node 17 : 8
Source to node 18 : 8
Source to node 19 : 8
Source to node 20 : 7
Source to node 21 : 8
Source to node 22 : 6
Source to node 23 : 7
Source to node 24 : 6
Source to node 25 : 6
Source to node 26 : 7
Source to node 27 : 8
Source to node 28 : 7
Source to node 29 : 7
Source to node 30 : 7
Source to node 31 : 7
Source to node 32 : 7
Source to node 33 : 8
Source to node 34 : 9
Source to node 35 : 7
Source to node 36 : 6
Source to node 37 : 7
Source to node 38 : 7
Source to node 39 : 8
Source to node 40 : 7
Source to node 41 : 7
S

Part (b) - Implementation of Dijkstra's algorithm using adjacency list + min heap as priority queue

For this implementation, we shall use the heapq module for our min heap.

In [44]:
def DijkstraB(G, source):
    time_start = timeit.default_timer()
    
    distance = [sys.maxsize] * G.V  # Original distance for all vertices are set to infinity
    path = [[] for i in range(G.V)]  # Initialize N number of sub-lists to store the path to each node
    S = [False] * G.V  # Initialize False to all the nodes since shortest path not yet discovered

    distance[source] = 0  # Set distance of source node to be zero
    
    minHeap = []
    for i in range(G.V):  # Insert all the vertices into min heap
        heapq.heappush(minHeap, [distance[i], i])
    
    while (len(minHeap) != 0):
        u = heapq.heappop(minHeap)
        S[u[1]] = True
        
        for elements in G.listgraph[u[1]]:
            if S[elements[0]] != True and distance[elements[0]] > distance[u[1]] + elements[1]:
                for z in minHeap:
                    if z[1] == elements[0]:
                        minHeap.remove(z)
                        heapq.heapify(minHeap)
                            
                distance[elements[0]] = distance[u[1]] + elements[1]
                    
                if source not in path[elements[0]]:
                    for nodes in path[u[1]]:
                        path[elements[0]].append(nodes)
                path[elements[0]].append(u[1])
                    
                heapq.heappush(minHeap, [distance[elements[0]], elements[0]])
    
    time_stop = timeit.default_timer()
    #END OF DIJKSTRA ALGORITHM
    
    #Print results
    print("Time elapsed for program: ", (time_stop - time_start)*1000, "milliseconds")  # Time in milliseconds
    print("Distance from source node,")
    for i in range(0, G.V):
        if distance[i] == sys.maxsize:
            print("Source to node", i, ": No such path exist")
        else:
            print("Source to node", i, ":", distance[i])
        resultB.append(distance[i])
    
    print()
    print("Path to get to each node,")
    
    for i in range(0, G.V):
        print("Path from source to node", i)
        print(path[i])
        print()

In [45]:
DijkstraB(g, 0)

Time elapsed for program:  7.21889999999803 milliseconds
Distance from source node,
Source to node 0 : 0
Source to node 1 : 5
Source to node 2 : 8
Source to node 3 : 7
Source to node 4 : 7
Source to node 5 : 7
Source to node 6 : 7
Source to node 7 : 8
Source to node 8 : 6
Source to node 9 : 7
Source to node 10 : 9
Source to node 11 : 7
Source to node 12 : 6
Source to node 13 : 8
Source to node 14 : 8
Source to node 15 : 7
Source to node 16 : 8
Source to node 17 : 8
Source to node 18 : 8
Source to node 19 : 8
Source to node 20 : 7
Source to node 21 : 8
Source to node 22 : 6
Source to node 23 : 7
Source to node 24 : 6
Source to node 25 : 6
Source to node 26 : 7
Source to node 27 : 8
Source to node 28 : 7
Source to node 29 : 7
Source to node 30 : 7
Source to node 31 : 7
Source to node 32 : 7
Source to node 33 : 8
Source to node 34 : 9
Source to node 35 : 7
Source to node 36 : 6
Source to node 37 : 7
Source to node 38 : 7
Source to node 39 : 8
Source to node 40 : 7
Source to node 41 : 7
So

Comparing the results of the 2 variations to see if they match

In [46]:
Error = False

for i in range(len(resultA)):
    if resultA[i] == resultB[i]:
        continue
    else:
        Error = True
        break
        
if (Error):
    print("Something went wrong")
    for i in range(len(resultA)):
        if resultA[i] != resultB[i]:
            print("Node", i, resultA[i] == resultB[i])
else:
    print("Output for both variations are the same")

Output for both variations are the same
